In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load data from CSV file
# DATA source : https://ourworldindata.org/grapher/car-sales?v=1&csvType=full&useColumnShortNames=false
file_path = '/Users/GURU/Desktop/ANALYSIS PROJECTS/Project 1/car-sales/car-sales.csv'  # Update this with the path to your CSV file

# Read the CSV file
df = pd.read_csv(file_path)

# Filter data for the required regions and years (2000-2023)
regions = ['Rest of World', 'United States', 'China', 'Europe']
df = df[(df['Year'] >= 2000) & (df['Entity'].isin(regions))]

# Check for missing or incorrect values
df.fillna(0, inplace=True)

# Calculate total sales for EV and Non-EV year-wise
yearly_ev_sales = df.pivot_table(index='Year', columns='Entity', values='Electric cars sold', aggfunc='sum').fillna(0) / 1_000_000  # Convert to millions
yearly_non_ev_sales = df.pivot_table(index='Year', columns='Entity', values='Non-electric car sales', aggfunc='sum').fillna(0) / 1_000_000  # Convert to millions

# Combine EV and Non-EV sales into a single DataFrame for each year
yearly_total_sales = pd.concat([yearly_ev_sales, yearly_non_ev_sales], axis=1)
yearly_total_sales.columns = [f'{col} EV Sales' for col in yearly_ev_sales.columns] + [f'{col} Non-EV Sales' for col in yearly_non_ev_sales.columns]

# Create an interactive stacked bar chart
fig = go.Figure()

for region in regions:
    fig.add_trace(go.Bar(
        x=yearly_total_sales.index,
        y=yearly_total_sales[f'{region} Non-EV Sales'],
        name=f'{region} Non-EV Sales',
        hovertemplate='Year: %{x}<br>Non-EV Sales: %{y:.2f}M',
    ))
    fig.add_trace(go.Bar(
        x=yearly_total_sales.index,
        y=yearly_total_sales[f'{region} EV Sales'],
        name=f'{region} EV Sales',
        hovertemplate='Year: %{x}<br>EV Sales: %{y:.2f}M',
    ))

fig.update_layout(
    barmode='stack',
    title={'text': 'Yearly Vehicle Sales by Region and Type (2000-2023)', 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top', 'font': {'size': 24, 'family': 'Arial', 'color': 'black', 'weight': 'bold'}},
    xaxis=dict(tickmode='linear', tick0=2000, dtick=1),
    yaxis=dict(title='Sales (in millions)'),
    legend_title=dict(text='Sales Type', font=dict(weight='bold')),
    hovermode='x unified'
)

fig.show()
